In [1]:
import pandas as pd
import json_lines
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_json('spiders/rosprav/5_130p.jl', lines=True)
df['title'] = df['title'].map(lambda x: x[0])
df['body']= df['body'].map(lambda x: x[0])

In [3]:
print('Количество судебных решений ', df.shape[0])

Количество судебных решений  2396


In [4]:
def parse_table(table_body):
    data = dict()
    for line in BeautifulSoup(table_body, 'lxml').find_all('tr'):
        data[line.find('td').get_text()] = line.find('th').get_text()
    return data

Выкидываем записи с пустой таблицей метаданных:

In [5]:
df = df.drop(df[df['table'].map(lambda x: len(x)==0)].index)

Парсим таблицу метаданных:

In [6]:
df['table'] = df['table'].map(lambda x: parse_table(x[0]))
df = df.join(df['table'].apply(pd.Series)).drop('table', axis=1)

Разделяем судебные решения словами `установил` <содержательная часть> `приговорил` <решение суда>

In [9]:
df_pr = df[df['body'].map(lambda text: any([phrase in re.sub(r"\s+", "", text.lower(), flags=re.UNICODE) for phrase in ['судприговорил', 'приговорил']]))].reset_index()

In [10]:
df_pr['body'] = df_pr['body'].map(lambda text: re.sub('у\s?с\s?т\s?а\s?н\s?о\s?в\s?и\s?л', 'установил', text.lower()))

In [11]:
df_pr = df_pr[df_pr.body.str.split('установил').map(len) == 2]

In [12]:
df_pr['body'] = df_pr['body'].str.split('установил')

In [13]:
df_pr['second'] = df_pr['body'].map(lambda x: re.sub('п\s?р\s?и\s?г\s?о\s?в\s?о\s?р\s?и\s?л', 'приговорил', x[1]))

In [14]:
df_pr = df_pr[df_pr['second'].str.split('приговорил').map(len) == 2]

In [15]:
df_pr['case'], df_pr['result'] = df_pr['second'].map(lambda text: text.replace(':','').split('приговорил')).str

Из решения суда выделяем номера статей кодекса:

In [ ]:
pattern = re.compile(r'(статьей|ст\.|статьи)\s*(\d+\.?\d*)')
df_pr['labels'] = df_pr['result'].map(lambda text:list(map(lambda x: float(x[1]), re.findall(pattern, text))))

In [17]:
df_pr = df_pr[df_pr['labels'].map(len) != 0]

In [18]:
df_pr['case'] = df_pr['case'].str.strip()

In [20]:
df_pr[['case', 'labels']].to_csv('parallel.csv', index=False)